In [120]:
from fourInRowGame import Chip, FourInRowGame
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import models
import torch
import numpy as np
import pickle
from tabulate import tabulate

In [83]:
pos_act = [1,3,5]
ac_Val = [0,3,2,6,5,2,1]
ac_Val
print([ac_Val[actions] for actions in pos_act])

[3, 6, 2]


In [84]:
print(action_values1[0])
av = np.copy(action_values1[0])
not_pos_act = mask = np.isin(np.arange(len(av)), pos_act, invert=True)
print(np.random.choice(np.flatnonzero(av == av.max())))
av[not_pos_act] = np.NINF
print(av)
print(np.random.choice(np.flatnonzero(av == av.max())))

[17.26745428 16.88747158 17.55294413 17.49255462 17.65974937 17.92912972
 17.92721583]
5
[       -inf 16.88747158        -inf 17.49255462        -inf 17.92912972
        -inf]
5


In [87]:
def create_board_states(nbrows,nbcolumns):   
    #boardstate is a list where all (x,y) are stored. The order is row major. It means that each line is concatainated.
    boardstate=[]
    count=0
    for y in range(nbrows):
        for x in range(nbcolumns):
            boardstate.append((x,y))
    return boardstate


def get_xy_from_agent_state(agentstate,boardstate):
    xy=boardstate[agentstate]
    return xy


def get_agent_state_from_xy(x,y,boardstate):
    return boardstate.index((x,y))


def get_possible_actions(env, ncol, nrow, terminated): 
    no_pos_action = True
    actions = np.array([])
    for col in range(ncol):
        if env.column_height(col) < nrow:
            actions = np.append(actions, np.array([col]))
            no_pos_action = False
    if no_pos_action:
        terminated = True
    return actions.astype(int), terminated


def get_next_state(board_now, board_before, boardstate):
    board_diff = board_now - board_before
    change = np.argwhere(board_diff != 0)
    next_state = get_agent_state_from_xy(change[0][1], change[0][0], boardstate)
    return(next_state)


def get_action_play(model, state):
    #Get predictions
    preds = model(state)
    weights = preds.clone().numpy(force=True)
    action = np.argmax(weights)
    return int(action), weights


def check_valid_play(action, weights, env, terminated):
    cont = True
    while cont:    
        if env.column_height(action) >= 6:
            if np.isfinite(weights).sum() != 0:
                weights[action] = np.NINF
                action = np.argmax(weights)
            else:
                return(None, terminated)
        else:
            cont = False    
    return action, terminated


def make_step_play(env, action, player, chip_player, terminated):
    # Model is player 1 and makes a step
    env.drop(chip_player, action)
    if env.check_for_victory():
        winner = player
        terminated = True
    else:
        not_full_cols = 0
        for col in range(6):
           if env.column_height(col) <= 5:
               not_full_cols += 1
        if not_full_cols == 0:
            terminated = True
        winner = 0
    return env, winner, terminated


def get_player_decision(env, player, terminated, state_simpleQ, action_values, model):
    # Save the board for Q-Learning
    board_before = env.get_simple_slots()
    # Player needs to move
    if player == "simpleQ":
        possible_actions, terminated = get_possible_actions(env, 7, 6, terminated)
        av = np.copy(action_values[state_simpleQ])
        not_possible_actions = np.isin(np.arange(len(av)), possible_actions, invert=True)
        av[not_possible_actions] = np.NINF
        action = np.random.choice(np.flatnonzero(av == av.max()))
    elif player == "deepQ":
        state_nn = torch.tensor(env.get_simple_slots(), dtype=torch.float)
        action, weights = get_action_play(model, state_nn)
        action, terminated = check_valid_play(action, weights, env, terminated)
    elif player == "rand":
        possible_actions, terminated = get_possible_actions(env, 7, 6, terminated)
        action = np.random.choice(possible_actions)
    else:
        raise ValueError(f"This type of player is not allowed")
    return action, terminated, board_before


In [89]:
def compare_q_learns(env, player1, player2, chip_p1, chip_p2, action_values1 = None, action_values2 = None, model1 = None, model2 = None):
    env.reset()
    terminated = False
    state_simpleQ = 0
    winner = 0
    boardstate = create_board_states(6, 7)
    while not terminated:
        # env.print()
        # print("\n")
        action, terminated, board_before = get_player_decision(env, player1, terminated, state_simpleQ, action_values1, model1)
        if terminated: 
            # env.print()
            # print("\n")
            return winner

        # Make the actual action:
        env, winner, terminated = make_step_play(env, action, 1, chip_p1, terminated)
        # Save the state for Simple Q-Learning
        state_simpleQ = get_next_state(env.get_simple_slots(), board_before, boardstate)

        # env.print()
        # print("\n")
        
        if terminated: 
            return winner
        if not terminated: 

            # Player 2 has to make a move
            action, terminated, board_before = get_player_decision(env, player2, terminated, state_simpleQ, action_values2, model2)
            if terminated: 
                # env.print()
                # print("\n")
                return winner
            # Make the actual action:
            env, winner, terminated = make_step_play(env, action, 2, chip_p2, terminated)
            # Save the state for Simple Q-Learning
            state_simpleQ = get_next_state(env.get_simple_slots(), board_before, boardstate)

    return winner

In [201]:
winner_board = [['player1', 'player2', 'rep', 'win1', 'win2', 'winX']]

In [206]:
with open('Q_Learn_av1.pkl', 'rb') as file:
    action_values1 = pickle.load(file)
with open('Q_Learn_av2.pkl', 'rb') as file:
    action_values2 = pickle.load(file)

In [144]:
model_type1 = "FC"
if "FC" in model_type1:
    in_dim1 = env.columns*env.rows
elif "CNN" in model_type1:
    in_dim1 = 1
else:
    raise ValueError(f"This model type is unknown to me.")
model1 = models.FCmedium(in_dim1, env.columns)
model1.load_state_dict(torch.load('FCmedium_1402_diff_ret'))
model1.eval()

FCmedium(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (net): Sequential(
    (0): Linear(in_features=42, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): ReLU()
    (8): Linear(in_features=16, out_features=7, bias=True)
  )
)

In [145]:
model_type2 = "CNN"
if "FC" in model_type2:
    in_dim2 = env.columns*env.rows
elif "CNN" in model_type2:
    in_dim2 = 1
else:
    raise ValueError(f"This model type is unknown to me.")
model2 = models.CNNmedium(in_dim2, env.columns)
model2.load_state_dict(torch.load('CNNmedium_1402_diff_ret'))
model2.eval()

CNNmedium(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (out): Linear(in_features=6, out_features=7, bias=True)
)

In [180]:
model_type3 = "CNN"
if "FC" in model_type3:
    in_dim3 = env.columns*env.rows
elif "CNN" in model_type3:
    in_dim3 = 1
else:
    raise ValueError(f"This model type is unknown to me.")
model3 = models.CNNmedium(in_dim3, env.columns)
model3.load_state_dict(torch.load('CNNmedium/Iteration_10'))
model3.eval()

CNNmedium(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (out): Linear(in_features=6, out_features=7, bias=True)
)

In [193]:
model_type4 = "FC"
if "FC" in model_type4:
    in_dim4 = env.columns*env.rows
elif "CNN" in model_type4:
    in_dim4 = 1
else:
    raise ValueError(f"This model type is unknown to me.")
model4 = models.FCmedium(in_dim4, env.columns)
model4.load_state_dict(torch.load('FCmedium/Iteration_10'))
model4.eval()

FCmedium(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (net): Sequential(
    (0): Linear(in_features=42, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): ReLU()
    (8): Linear(in_features=16, out_features=7, bias=True)
  )
)

In [210]:
model_type5 = "CNN"
if "FC" in model_type5:
    in_dim5 = env.columns*env.rows
elif "CNN" in model_type5:
    in_dim5 = 1
else:
    raise ValueError(f"This model type is unknown to me.")
model5 = models.CNNmedium(in_dim5, env.columns)
model5.load_state_dict(torch.load('CNNmedium_1302'))
model5.eval()

CNNmedium(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (out): Linear(in_features=6, out_features=7, bias=True)
)

In [236]:
model_type6 = "FC"
if "FC" in model_type6:
    in_dim6 = env.columns*env.rows
elif "CNN" in model_type6:
    in_dim6 = 1
else:
    raise ValueError(f"This model type is unknown to me.")
model6 = models.FCsmall(in_dim6, env.columns)
model6.load_state_dict(torch.load('FCsmall_10_02_2024'))
model6.eval()

FCsmall(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (net): Sequential(
    (0): Linear(in_features=42, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=7, bias=True)
  )
)

In [237]:
model_type7 = "CNN"
if "FC" in model_type7:
    in_dim7 = env.columns*env.rows
elif "CNN" in model_type7:
    in_dim7 = 1
else:
    raise ValueError(f"This model type is unknown to me.")
model7 = models.CNNsmall(in_dim7, env.columns)
model7.load_state_dict(torch.load('CNNsmall_10_02_2024'))
model7.eval()

CNNsmall(
  (net): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 1, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (out): Linear(in_features=6, out_features=7, bias=True)
)

In [203]:
env = FourInRowGame(6,7)

In [244]:
nn_names = ["FCmedium; Diff Ret", "CNNmedium; Diff Ret", "FCmedium; Diff Loss", "CNNmedium; Diff Loss", "CNNmedium; First", "FCsmall", "CNNsmall"]
player1 = "rand"
player2 = "deepQ"
model_name2 = nn_names[6]
# model_name2 = nn_names[1]
chip_p1 = Chip.RED
chip_p2 = Chip.YELLOW
repetitions = 1000

In [245]:
win1_counter = 0
win2_counter = 0
winX_counter = 0
for reps in range(repetitions):
    winner = compare_q_learns(env, player1, player2, chip_p1, chip_p2, model2 = model7)
    if winner == 1:
        win1_counter += 1
    elif winner == 2:
        win2_counter += 1
    elif winner == 0:
        winX_counter += 1
    else:
        raise ValueError(f"This winner is impossible")
if player1 == "deepQ":
    player1 = player1 + "; " + model_name1 
if player2 == "deepQ":
    player2 = player2 + "; " + model_name2
winner_board.append([player1, player2, repetitions, win1_counter/repetitions, win2_counter/repetitions, winX_counter/repetitions])

In [246]:
table = tabulate(winner_board[1:], winner_board[0], tablefmt = "grid")

In [248]:
with open('WinnerBoard_1602v2.pkl', 'wb') as file:
    pickle.dump(table, file)

In [247]:
print(table)

+-----------------------------+-----------------------------+-------+--------+--------+--------+
| player1                     | player2                     |   rep |   win1 |   win2 |   winX |
+=============================+=============================+=======+========+========+========+
| simpleQ                     | rand                        |  1000 |  0.78  |  0.219 |  0.001 |
+-----------------------------+-----------------------------+-------+--------+--------+--------+
| rand                        | simpleQ                     |  1000 |  0.417 |  0.58  |  0.003 |
+-----------------------------+-----------------------------+-------+--------+--------+--------+
| deepQ; FCmedium; Diff Ret   | rand                        |  1000 |  0.828 |  0.172 |  0     |
+-----------------------------+-----------------------------+-------+--------+--------+--------+
| rand                        | deepQ; FCmedium; Diff Ret   |  1000 |  0.211 |  0.789 |  0     |
+-----------------------------